<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/channel_extraction_and_drainage_area_examples/get_channel_heads_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Channel head extraction example

Notebook last updated by Simon M Mudd 09/05/2023

In this notebook we will extract channel heads using methods described in:

* Clubb, F.J., Mudd, S.M., Milodowski, D.T., Hurst, M.D., Slater, L.J., 2014. Objective extraction of channel heads from high-resolution topographic data. Water Resources Research 50, 4283–4304. https://doi.org/10.1002/2013WR015167
* Grieve, S.W.D., Mudd, S.M., Hurst, M.D., Milodowski, D.T., 2016. A nondimensional framework for exploring the relief structure of landscapes. Earth Surface Dynamics 4, 309–325. https://doi.org/10.5194/esurf-4-309-2016

These methods are used with high resolution data. 

The Grieve et al 2016 paper found that channel head extraction is unliklely to succeed if the grid spacing is greater than 10 metres. 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`:

In [ ]:
!pip install lsdviztools  &> /dev/null

## Add some necessary packages

First check the version of lsdviztools. **For this notebook we need lsdviztools >=0.4.9**

In [ ]:
import lsdviztools
lsdviztools.__version__

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

### Get the example data

The example data is an 18 Mb topographic dataset which you need to download:

In [ ]:
# Get the data from github
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChannelExtractionData/IndianCreek_1m/indian_creek.hdr", "indian_creek.hdr")
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChannelExtractionData/IndianCreek_1m/indian_creek.bil", "indian_creek.bil")

Lets make sure that file (`indian_creek.bil` and its header) have been downloaded:

In [ ]:
!ls indian_creek*

## Look at the topography

Lets start by looking at the topography. The ```"remove_seas" : "true"``` deals with awkward nodata around the edge of the DEM:

In [ ]:
lsdtt_parameters = {"remove_seas" : "true",
                    "write_hillshade" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
### Plot the hillshade
%matplotlib inline
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")

We can get the channel sources using using `lsdtt-channel-extraction`. We will use the `print_wiener_channels` option. You also need to get the sources with the option `print_sources_to_csv`. This will take around a minute. 

In [ ]:
lsdtt_parameters = {"print_wiener_channels" : "true",
                    "print_sources_to_csv" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-channel-extraction", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Right, now that that has finished, lets see what files we got:

In [ ]:
!ls indian_creek*

There are actually 2 files with sources here, one is `indian_creek_ATsources.csv` and the other is `indian_creek_Wsources.csv`. `AT` stands for *area threshold* (and is produced by default) and `W` is for wiener. 

## Look at the Sources using folium

We can plot these sources on a folium map to compare them.
Before we do that though, we need to load the two datasets with pandas

In [ ]:
import pandas as pd

sources_AT_df = pd.read_csv("indian_creek_ATsources.csv")
sources_W_df = pd.read_csv("indian_creek_Wsources.csv")

In [ ]:
# This is for the area threshold points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_AT_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

In [ ]:
# This is for the wiener points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_W_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

You will notice that the Wiener sources appear at much more realistic locations on this map. If you want to use channel sources and are particular about the headwaters, you probably want to use this method (see https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2013WR015167 and https://esurf.copernicus.org/articles/4/627/2016/).

When looking for valleys and floodplains, however, you can set a minimum stream order so that the smallest channels are not captured. Even if the area threshold channel sources are not particularly accurate, the higher order streams will still be in reasonable locations. 

## Plot the points using `lsdviztools`

We can also plot these points using one of the `lsdviztools` plotting routines:

In [ ]:
sources_W_df.head()

In [ ]:
### Plot the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "drainage_area", 
                                          scaled_data_in_log = True,
                                          points_fname = "indian_creek_Wsources.csv", 
                                          scale_points = False, manual_size =10,
                                          cmap="cubehelix", save_fig=False, size_format="geomorphology")

## Extract a channel network on the basis of these source points

In [ ]:
lsdtt_parameters = {"remove_seas" : "true",
                    "print_channels_to_csv" : "true",
                    "CHeads_file" : "indian_creek_Wsources.csv",
                    "use_extended_channel_data" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-channel-extraction", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Lets see what csv file this produced:
In this code an area threshold channel network is created by default (`indian_creek_AT_CN.csv`) but in addition there is a channel network from the channel head file (`indian_creek_FromCHF_CN.csv`)

In [ ]:
!ls *.csv

Right, lets load that channel file into a pandas dataframe so we can see what it looks like:

In [ ]:
import pandas as pd
df = pd.read_csv("indian_creek_FromCHF_CN.csv")
df.head()

We need to list the column headers to make sure we have the exact column names:

In [ ]:
list(df)

Okay, now we can do some fancy formatting and plot the channel network, with the points sized by the stream order. 

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "elevation(m)", 
                                          points_fname = "indian_creek_FromCHF_CN.csv", 
                                          scale_points = True, column_for_scaling = "Stream Order",
                                          scaled_data_in_log = False, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=False, size_format="geomorphology")